<a href="https://colab.research.google.com/github/zoe-zhang99/interactive_dashboard_plotly/blob/main/dashboard-with-figures-and-table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Due to data confidientiality, I cannot provide the data source or show any figure here. This is only to show how the code was written.

# Package install and import

In [ ]:
!pip install pingouin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import pingouin as pg

import plotly.express as px
import plotly.graph_objects as go

# Data preprocessing

This step has already filtered out participants who did not leave an email address, which is at the very beginning of the survey

In [ ]:
# define a function for data preprocessing
def clean_text(df, year):
  '''
  please input df as such: 'CAB_Baseline_and_follow-up_Year1_202004_May17_2021_11.01.csv'
  please input year as such: 2021
  '''
  # read file
  df = pd.read_csv(df)

  # cut extra rows
  df = df.iloc[2:]

  # replace blanks with NA
  df = df.replace(r'^\s*$', np.nan, regex=True)

  # filter out responses without email input
  df = df[df.emailinoput.notnull()] # comment this out if you need all the entries

  # create new column recording survey year
  df['year'] = year
  
  # drop columns with all null values
  df = df.dropna(axis=1, how='all')

  # return df
  return df

In [ ]:
# define a function for data preprocessing [the first two rows have already been cut in R]
def clean_num(df, year):
  '''
  please input df as such: 'CAB_Baseline_and_follow-up_Year1_202004_May17_2021_11.01.csv'
  please input year as such: 2021
  '''
  # read file
  df = pd.read_csv(df)

  # replace blanks with NA
  df = df.replace(r'^\s*$', np.nan, regex=True)

  # filter out responses without email input
  df = df[df.emailinoput.notnull()] # comment this out if you need all the entries

  # create new column recording survey year
  df['year'] = year
  
  # drop columns with all null values
  df = df.dropna(axis=1, how='all')

  # return df
  return df

read and clean dataset, please change it to other file names if you download other files

In [ ]:
df1 = clean_text('CAB_2020_text.csv',2020)
df2 = clean_text('CAB_2021_text.csv',2021)
df3 = clean_text('CAB_2022_text.csv',2022)

convert zip from 2020 data to state

In [ ]:
df1['zip'] = df1['zip'].apply(pd.to_numeric)

In [ ]:
def conditions(df):
  if (df['zip'] >= 66002) and (df['zip'] <= 67954):
    return 'Kansas'
  if (df['zip'] >= 60001) and (df['zip'] <= 62999):
    return 'Illinois'
  if (df['zip'] >= 46001) and (df['zip'] <= 47997):
    return 'Indiana'
  if (df['zip'] >= 70001) and (df['zip'] <= 71497):
    return 'Indiana'
  if (df['zip'] >= 6001) and (df['zip'] <= 6928):
    return 'Connecticut'
  if (df['zip'] >= 24701) and (df['zip'] <= 26886):
    return 'West Virginia'
  if (df['zip'] >= 37010) and (df['zip'] <= 38589):
    return 'Tennessee'
  if (df['zip'] >= 57001) and (df['zip'] <= 57799):
    return 'South Dakota'
  if (df['zip'] >= 43001) and (df['zip'] <= 45999):
    return 'Ohio'
  if (df['zip'] >= 7001) and (df['zip'] <= 8989):
    return 'New Jersey'
  if (df['zip'] >= 48001) and (df['zip'] <= 49971):
    return 'Michigan'
  if (df['zip'] >= 40003) and (df['zip'] <= 42788):
    return 'Kentucky'
  if (df['zip'] >= 63001) and (df['zip'] <= 65899):
    return 'Missouri'
  if (df['zip'] >= 27006) and (df['zip'] <= 28909):
    return 'North Carolina'
  if (df['zip'] >= 15001) and (df['zip'] <= 19640):
    return 'Pennsylvania'
  if (df['zip'] >= 30001) and (df['zip'] <= 31999) or (df['zip'] == 39901):
    return 'Georgia'
  if (df['zip'] >= 38601) and (df['zip'] <= 39776) or (df['zip'] == 71233):
    return 'Mississippi'
  if (df['zip'] >= 22001) and (df['zip'] <= 24658) and (df['zip'] == 20040 or 20041 or 20041 or 20167):
    return 'Virginia'
  else:
    return 'N/A'

In [ ]:
df1['state'] = df1.apply(conditions, axis=1)

the final dataset (text value)

In [ ]:
# outer join the dfs
result = pd.concat([df1, df2, df3], join="outer")
result['state'] = result['state'].fillna('N/A')

the final dataset (numerical value)

In [ ]:
df1n = clean_num('year1_2022-08-17.csv', 2020)
df2n = clean_num('year2_2022-08-09.csv', 2021)
df3n = clean_num('year3_2022-08-17.csv', 2022)

In [ ]:
result_n = pd.concat([df1n, df2n, df3n], join="outer")
num = result_n.select_dtypes([np.number])
num = num.rename(columns = {'state':'state_id'})
num['state'] = result['state'].values.tolist()

# Dataframe

- survey: Survey Completion Status
- demo: Demographic Distribution of Participants
- measurement: Different Key Measurement (measurement1 from text responses, measurement2 from numerical responses)
- trend: Yearly trend for certain items
- reliability measure: Croabach's Alpha for different sets of questions


### regroup certain columns

In [ ]:
# convert to numerical columns
result[['Progress','age','hoursvol','netoutorg_1','netorgnum_1','internetfreq']] = result[['Progress','age','hoursvol','netoutorg_1','netorgnum_1','internetfreq']].apply(pd.to_numeric)

In [ ]:
# regroup columns
result['Progress_group'] = pd.cut(result['Progress'],
                             bins=[0,50,80,99,100],
                             labels=['0-50', '50-80', '80-99','100'])
result['age_group'] = pd.cut(result['age'],
                             bins=[18,25,40,55,70,100],
                             labels=['18-25', '25-40', '40-55', '55-70','70+'])
result['hoursvol_group'] = pd.cut(result['hoursvol'],
                             bins=[0,5,10,15,20,50,10000000000],
                             labels=['0-5', '5-10', '10-15', '15-20','20-50','50+'])
result['netoutorg_group'] = pd.cut(result['netoutorg_1'],
                             bins=[0,5,10,15,20,50,10000000000],
                             labels=['0-5', '5-10', '10-15', '15-20','20-50','50+'])
result['netorgnum_group'] = pd.cut(result['netorgnum_1'],
                             bins=[0,5,10,15,20,50,10000000000],
                             labels=['0-5', '5-10', '10-15', '15-20','20-50','50+'])
result['internetfreq_group'] = pd.cut(result['internetfreq'],
                             bins=[0,10,20,30,40,50,60,70,80,100],
                             labels=['0-10', '10-20', '20-30', '30-40','40-50','50-60','60-70','70-80','80+'])

### Survey figure

This shows the survey completion status

In [ ]:
# select columns
survey = result[['Progress_group','Finished']]
survey = survey.apply(lambda x: x.sort_values().values)

In [ ]:
# note for annotation
note1 = 'Indicator of progress of the survey (%)'
note2 = 'If the participant has finished the survey'

anno1 = [dict(
    showarrow=False,
    text=note1,
    font=dict(size=12), 
    xref='paper',
    x=1,
    yref='paper',
    y=1.1
    )]

anno2 = [dict(
    showarrow=False,
    text=note2,
    font=dict(size=12),
    xref='paper',
    x=1,
    yref='paper',
    y=1.1
    )]

In [ ]:
# Initialize figure
fig1 = go.Figure()

buttons = []

for col_name in survey.columns:
  ## add traces
  fig1.add_trace(go.Bar(
              x=survey[col_name].unique(),
              y=survey[col_name].value_counts(),
              text=survey[col_name].value_counts(),  
              name=col_name,
              visible = False
              )
          )

fig1.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=list([
                dict(label="Progress_group",
                     method="update",
                     args=[{"visible": [True, False]},
                           {"annotations": anno1}]),
                dict(label="Finished",
                     method="update",
                     args=[{"visible": [False, True]},
                           {"annotations": anno2}])
            ]),
             type="dropdown",
             x = 1,
             y = 1.1,
             xanchor = 'right',
             yanchor = 'bottom'
        )     
    ],
    margin=dict(l=20, r=20, t=60, b=20),#setup the margin
    paper_bgcolor="aliceblue", #setup the background color
    title_text="Survey Completion Status<br><br>",
    title_x=0.5,
    showlegend=False,
    title_font_size=22)

fig1.show()

### Participant figure

This displays the demographic information of the participants

In [ ]:
# select columns
demo = result[['join_yr', 'inst', 'membdrug', 'othboar', 'leadoth', 'edu','hisp','race','sex',
               'age_group', 'hoursvol_group', 'netorgnum_group','netoutorg_group','internetfreq_group',
               'cabsat_1','cabsat_2','cabsat_3', 'cabactivity_1','cabactivity_2', 'cabactivity_3' ]]
demo = demo.apply(lambda x: x.sort_values().values)

In [ ]:
# note for annotation
note1 = 'The first year the participants joined CAB' # join_yr
note2 = 'Institutions the participants work with' # inst
note3 = 'Are you a member of a drug board or coalition?' # membdrug
note4 = 'Are you a member of other community boards?' # othboar
note5 = 'Are you a leader of other community boards?' # leadoth
note6 = 'Education level' #edu
note7 = 'If hispanic or not' # hisp
note8 = 'Race/Ethnicity' # race
note9 = 'Sex/Gender' # sex
note10 = 'Age group' # age_group
note11 = 'How many hours the participants would be willing to volunteer duriing this year' # hoursvol_group
note12 = 'How many organizations WITHIN the county do the participants typically work with' # netorgnum_group
note13 = 'How many organizations OUTSIDE the county do the participants typically work with' # netoutorg_group
note14 = 'How many hours a week do the participants use the Internet' # internetfreq_group
note15 = 'Satisfaction with being involved in CAB' # cabsat_1
note16 = 'Satisfaction with Trimonthly Meetings' # cabsat_2
note17 = 'Satisfaction with frequency of contacts (# of emails and phone calls you are currently receiving from us) ' # cabsat_3
note18 = 'I feel that I\'m knowledgeable about the goals of CAB' # cabactivity_1
note19 = 'I feel that I\'m knowledgeable about the activities of CAB' # cabactivity_2
note20 = 'I feel that I need more information about the CAB in general' # cabactivity_3

In [ ]:
note = [note1, note2, note3, note4, note5, note6, note7, note8, note9, note10, note11, note12, note13, note14, note15, note16, note17, note18, note19, note20]

d = dict()

#add note to annotation
for idx, value in enumerate(note):
    key = 'anno' + str(idx)
    d[key] = [dict(
    showarrow=False,
    text=value,
    font=dict(size=12), 
    xref='paper',
    x=1,
    yref='paper',
    y=1.1,
    align = 'right'
    )]

In [ ]:
# Initialize figure
fig2 = go.Figure()

buttons2 = []

for col_name in demo.columns:
  ## add traces
  fig2.add_trace(go.Bar(
            x=demo[col_name].unique(),
            y=demo[col_name].value_counts(), 
            text=demo[col_name].value_counts(), 
            name=col_name,
            visible=False
            )
        )

for i, column in enumerate(demo.columns):
  args = [False] * len(demo.columns)
  args[i] = True

  button2 = dict(label = column,
                  method = "update",
                  args=[{"visible": args},
                        {"annotations": list(d.values())[i]}])
  buttons2.append(button2)

fig2.update_layout(
    title_text="Participant Information",
    title_x=0.5,
    showlegend=False,
    title_font_size=22
) 

fig2.update_layout(
    updatemenus=[dict(active=0,
                      type="dropdown",
                    buttons=buttons2,
                    x = 1,
                    y = 1.1,
                    xanchor = 'right',
                    yanchor = 'bottom'
                )], 
    margin=dict(l=20, r=20, t=60, b=20),#setup the margin
    paper_bgcolor="aliceblue", #setup the background color
)

fig2.show()

### Key performance variables figure

This displays the data distribution of some key questions asked each year

In [ ]:
# get text response measurement
measurement1 = result.loc[:, result.columns.str.contains("issuehelp_|cdrugprob_|trust_|cad_")].drop(['issuehelp_14_TEXT', 'issuehelp_14_TEXT_TEXT'], axis=1)
measurement1[['year','state']] = result[['year','state']].values.tolist()
measurement1 = measurement1.reset_index().drop(['index'], axis =1).replace(['Click to write Scale Point 2', 'Click to write Scale Point 3', 'Click to write Scale Point 4'], 'Not Applicable')

# get numerical response measurement
measurement2 = num.loc[:, num.columns.str.contains("_avg|_ttl")]
measurement2[['year','state']] = num[['year','state']].values.tolist()
measurement2 = measurement2.reset_index().drop(['index'], axis =1)

# concat two responses for plotting
measurement = pd.concat([measurement1, measurement2], axis = 1)
measurement = measurement.loc[:, ~measurement.columns.duplicated()]

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# <br> means new line in plotly annotation
# ordered by column
note2 = ['issues pressing to the community where you live - Accessing healthcare for 2020 & 2021, Accessing in-person healthcare (starting 2022)<br>1=Community working to help, 2=Government working to help, 0=Not helping, 4=Not an issue, so irrelevant',# issuehelp_1
        'issues pressing to the community where you live - Economy (whether it is problems with unemployment, housing, etc)<br>1=Community working to help, 2=Government working to help, 0=Not helping, 4=Not an issue, so irrelevant', # issuehelp_2
        'issues pressing to the community where you live - Education<br>1=Community working to help, 2=Government working to help, 0=Not helping, 4=Not an issue, so irrelevant', # issuehelp_3
        'issues pressing to the community where you live - Violence, including domestic abuse<br>1=Community working to help, 2=Government working to help, 0=Not helping, 4=Not an issue, so irrelevant', # issuehelp_4
        'issues pressing to the community where you live - Drugs, including misuse of prescriptions or any kind of drugs<br>1=Community working to help, 2=Government working to help, 0=Not helping, 4=Not an issue, so irrelevant', # issuehelp_5
        'issues pressing to the community where you live - Crime<br>1=Community working to help, 2=Government working to help, 0=Not helping, 4=Not an issue, so irrelevant', # issuehelp_6
        'issues pressing to the community where you live - Mental health<br>1=Community working to help, 2=Government working to help, 0=Not helping, 4=Not an issue, so irrelevant', # issuehelp_7
        'issues pressing to the community where you live - Homelessness<br>1=Community working to help, 2=Government working to help, 0=Not helping, 4=Not an issue, so irrelevant', # issuehelp_8
        'issues pressing to the community where you live - Prejudice against people of different ethnic or racial groups<br>1=Community working to help, 2=Government working to help, 0=Not helping, 4=Not an issue, so irrelevant', # issuehelp_9
        'issues pressing to the community where you live - Prejudice against LGBTQ+ people<br>1=Community working to help, 2=Government working to help, 0=Not helping, 4=Not an issue, so irrelevant', # issuehelp_10
        'issues pressing to the community where you live - Depression, hopelessness, or despair<br>1=Community working to help, 2=Government working to help, 0=Not helping, 4=Not an issue, so irrelevant', # issuehelp_11
        'issues pressing to the community where you live - Use of alcohol<br>1=Community working to help, 2=Government working to help, 0=Not helping, 4=Not an issue, so irrelevant', # issuehelp_12
        'issues pressing to the community where you live - Deteriorating traditional family values<br>1=Community working to help, 2=Government working to help, 0=Not helping, 4=Not an issue, so irrelevant', # issuehelp_13
        'A key issue not listed in the survey (needs to check the free text response)', # issuehelp_14 (free text response)
        'issues pressing to the community where you live - Accessing online healthcare (starting 2022)<br>1=Community working to help, 2=Government working to help, 0=Not helping, 4=Not an issue, so irrelevant', # issuehelp_15
        'issues pressing to the community where you live - Prejudice against people who use drugs (starting 2022)<br>1=Community working to help, 2=Government working to help, 0=Not helping, 4=Not an issue, so irrelevant', # issuehelp_16
        'issues pressing to the community where you live - Misleading/incorrect health information (starting 2022)<br>1=Community working to help, 2=Government working to help, 0=Not helping, 4=Not an issue, so irrelevant', # issuehelp_17
         
        'Marijuana ("pot", "weed")<br>0=Not common in my community, 1=Common in my community', # cdrugprob_1
        'Amphetamines (prescription stimulants like Ritalin, Adderal)<br>0=Not common in my community, 1=Common in my community', # cdrugprob_2
        'Methamphetamine ("speed", "meth")<br>0=Not common in my community, 1=Common in my community', # cdrugprob_3
        'Cocaine<br>0=Not common in my community, 1=Common in my community', # cdrugprob_4
        'Prescription Opioids for pain (like Oxycontin, Vicodin, Norco, Percocet, others)<br>0=Not common in my community, 1=Common in my community', # cdrugprob_5
        'Heroin<br>0=Not common in my community, 1=Common in my community', # cdrugprob_6
        'Fentanyl<br>0=Not common in my community, 1=Common in my community', # cdrugprob_7
        'Non-opioid prescription pills for pain (like gabapentin [Neurontin], duloxetine [Cymbalta], venlafaxine [Effexor])<br>0=Not common in my community, 1=Common in my community', # cdrugprob_8
        'Hallucinogens ("LSD", "psilocybin mushrooms", "DMT")<br>0=Not common in my community, 1=Common in my community', # cdrugprob_9
        'Thinner orr other inhalents or solvents, suchas glue<br>0=Not common in my community, 1=Common in my community', # cdrugprob_10
        'GHB<br>0=Not common in my community, 1=Common in my community', # cdrugprob_11
        'Prescription pills for anxiety (like diazepam [Valium], lorazepam, alprazolam)<br>0=Not common in my community, 1=Common in my community', # cdrugprob_12
        'Cigarettes<br>0=Not common in my community, 1=Common in my community', # cdrugprob_13
        'Chewing Tobacco<br>0=Not common in my community, 1=Common in my community', # cdrugprob_14
        'Nicotine Vaping products<br>0=Not common in my community, 1=Common in my community', # cdrugprob_15
        'THC (psychoactive compound of Marijuana) Vaping products<br>0=Not common in my community, 1=Common in my community', # cdrugprob_16
        'Ketamine (starting 2022)<br>0=Not common in my community, 1=Common in my community', # cdrugprob_17
         
        'organizations people in your community place the most trust in - Local government<br>1=No trust at all, 2=Very little trust, 3=They are neutral, 4=Some trust, 5=A great deal of trust', # trust_1
        'organizations people in your community place the most trust in - Healthcare providers<br>1=No trust at all, 2=Very little trust, 3=They are neutral, 4=Some trust, 5=A great deal of trust', # trust_2
        'organizations people in your community place the most trust in - Federal government<br>1=No trust at all, 2=Very little trust, 3=They are neutral, 4=Some trust, 5=A great deal of trust', # trust_3
        'organizations people in your community place the most trust in - Federal government efforts to control and prevent the spread of disease, such as the Centers for Disease Control (CDC)<br>1=No trust at all, 2=Very little trust, 3=They are neutral, 4=Some trust, 5=A great deal of trust', # trust_4
        'organizations people in your community place the most trust in - Religious organizations in the community<br>1=No trust at all, 2=Very little trust, 3=They are neutral, 4=Some trust, 5=A great deal of trust', # trust_5
        'The types of sources and organizations that people in your community place the most trust in - Community organizations<br>1=No trust at all, 2=Very little trust, 3=They are neutral, 4=Some trust, 5=A great deal of trust', # trust_6
         
        'Most believe that people who have a substance use disorder are responsible for their problems<br>1=Strongly disagree, 2=Somewhat disagree, 3=Neither agree nor disagree, 4=Somewhat agree, 5=Strongly agree', # cad_1
        'Most believe that people develop a substance use disorder because of their own choices<br>1=Strongly disagree, 2=Somewhat disagree, 3=Neither agree nor disagree, 4=Somewhat agree, 5=Strongly agree', # cad_2
        'Most believe that people develop a substance use disorder because of circumstances beyond their control<br>1=Strongly disagree, 2=Somewhat disagree, 3=Neither agree nor disagree, 4=Somewhat agree, 5=Strongly agree', # cad_3
        
        'Calculate the total number of issues<br>1=Community working to help, 2=Government working to help, 0=Not helping, 4=Not an issue, so irrelevant', # issuehelp_ttl
        'Calculate the total number of drug problem<br>0=Not common in my community, 1=Common in my community', # cdrugprob_ttl
        'Calculate the level of trust to organizations<br>1=No trust at all, 2=Very little trust, 3=They are neutral, 4=Some trust, 5=A great deal of trust', # trust_avg
        'Calculate the level of attitude toward drug use in the community<br>1=Strongly disagree, 2=Somewhat disagree, 3=Neither agree nor disagree, 4=Somewhat agree, 5=Strongly agree', # attdruguse_avg
        'Calculate the level of stigma of people who use drugs<br>1=Strongly disagree, 2=Somewhat disagree, 3=Neither agree nor disagree, 4=Somewhat agree, 5=Strongly agree', #stigma_avg
        'Calculate the level of alienation of people who use drugs<br>1=Strongly disagree, 2=Somewhat disagree, 3=Neither agree nor disagree, 4=Somewhat agree, 5=Strongly agree', #alienation_avg
        'Calculate the level of community perceptions of people who use drugs<br>1=Strongly disagree, 2=Somewhat disagree, 3=Neither agree nor disagree, 4=Somewhat agree, 5=Strongly agree', #deval_avg
        'Calculate the level of community perceptions of stigma of services for people who use drugs<br>1=Strongly disagree, 2=Somewhat disagree, 3=Neither agree nor disagree, 4=Somewhat agree, 5=Strongly agree', #pwudserv_avg
        'Calculate the level of community perceptions of recovery for people who use drugs<br>1=Strongly disagree, 2=Somewhat disagree, 3=Neither agree nor disagree, 4=Somewhat agree, 5=Strongly agree' #pwudrecovery_avg
        ]

d2 = dict()

# update for annotation
for idx, value in enumerate(note2):
    key = 'anno' + str(idx)
    d2[key] = [dict(
    showarrow=False,
    text=value,
    font=dict(size=12), 
    xref='paper',
    x=1,
    yref='paper',
    y=1.1,
    align = 'right'
    )]

In [ ]:
fig3 = go.Figure()

buttons3 = []

for col_name in measurement1.set_index(['year','state']).columns.to_list():
  ## add traces
  fig3.add_trace(go.Pie(
            values=measurement1.set_index(['year','state'])[col_name].value_counts(), 
            labels=measurement1.set_index(['year','state'])[col_name].value_counts().index.tolist(),
            textinfo='label+percent',
            visible=False,
            )
  )

for col_name in measurement2.set_index(['year','state']).columns:
  ## add traces
  fig3.add_trace(go.Box(
            y=measurement2.set_index(['year','state'])[col_name].values.tolist(),
            text=measurement2.set_index(['year','state'])[col_name].values.tolist(),  
            name=col_name,
    boxpoints='all',
    jitter=0.3,
    marker = dict(
        color = 'rgb(214,12,140)',
    ),
            visible=False
            )
        )

for i, column in enumerate(measurement.set_index(['year','state']).columns):
  args = [False] * len(measurement.set_index(['year','state']).columns)
  args[i] = True

  button3 = dict(label = column,
                  method = "update",
                  args=[{"visible": args},
                        {"annotations": list(d2.values())[i]}])
  buttons3.append(button3)

fig3.update_layout(
    title_text="Key Measurement",
    title_x=0.5,
    showlegend=False,
    title_font_size=22
) 

fig3.update_layout(
    updatemenus=[dict(active=0,
                      type="dropdown",
                    buttons=buttons3,
                    x = 1,
                    y = 1.1,
                    xanchor = 'right',
                    yanchor = 'bottom'
                )], 
    margin=dict(l=20, r=20, t=60, b=20),#setup the margin
    paper_bgcolor="aliceblue", #setup the background color
)

fig3.show()

### Yearly trend

This displays the yearly trend for survey items

In [ ]:
# select columns
trend = num[['Status','issuehelp_ttl', 'cdrugprob_ttl', 'trust_avg', 'attdruguse_avg', 'year', 'stigma_avg', 'alienation_avg', 'deval_avg', 'pwudserv_avg', 'pwudrecovery_avg']]
trend = trend.rename({'Status':'num_of_participants'}, axis= 1)

In [ ]:
# update the note for each dropdown button
# ordered by each column

note4 = ['Number of Participants',
         'Calculate the total number of issues<br>1=Community working to help, 2=Government working to help, 0=Not helping, 4=Not an issue, so irrelevant', # issuehelp_ttl
        'Calculate the total number of drug problem<br>0=Not common in my community, 1=Common in my community', # cdrugprob_ttl
        'Calculate the level of trust to organizations<br>1=No trust at all, 2=Very little trust, 3=They are neutral, 4=Some trust, 5=A great deal of trust', # trust_avg
        'Calculate the level of attitude toward drug use in the community<br>1=Strongly disagree, 2=Somewhat disagree, 3=Neither agree nor disagree, 4=Somewhat agree, 5=Strongly agree', # attdruguse_avg
        '(starting 2022) Calculate the level of stigma of people who use drugs<br>1=Strongly disagree, 2=Somewhat disagree, 3=Neither agree nor disagree, 4=Somewhat agree, 5=Strongly agree', #stigma_avg
        '(starting 2022) Calculate the level of alienation of people who use drugs<br>1=Strongly disagree, 2=Somewhat disagree, 3=Neither agree nor disagree, 4=Somewhat agree, 5=Strongly agree', #alienation_avg
        '(starting 2022) Calculate the level of community perceptions of people who use drugs<br>1=Strongly disagree, 2=Somewhat disagree, 3=Neither agree nor disagree, 4=Somewhat agree, 5=Strongly agree', #deval_avg
        '(starting 2022) Calculate the level of community perceptions of stigma of services for people who use drugs<br>1=Strongly disagree, 2=Somewhat disagree, 3=Neither agree nor disagree, 4=Somewhat agree, 5=Strongly agree', #pwudserv_avg
        '(starting 2022) Calculate the level of community perceptions of recovery for people who use drugs<br>1=Strongly disagree, 2=Somewhat disagree, 3=Neither agree nor disagree, 4=Somewhat agree, 5=Strongly agree' #pwudrecovery_avg
        ]

d4 = dict()

for idx, value in enumerate(note4):
    key = 'anno' + str(idx)
    d4[key] = [dict(showarrow=False,
                    text=value,
                    font=dict(size=12), 
                    xref='paper',
                    x=1,
                    yref='paper',
                    y=1.1,
                    align = 'right'
                    )]

In [ ]:
# Initialize figure
fig4 = go.Figure()

buttons4 = []

for col_name in trend.set_index('year').columns:
  ## add traces
  fig4.add_trace(go.Scatter(
            x=[str(x) for x in trend.year.unique().tolist()],
            # add different aggregations to the figure
            y=trend.groupby('year').agg({'num_of_participants':'count', 
                         'issuehelp_ttl':'mean', 
                         'cdrugprob_ttl':'mean', 
                         'trust_avg':'mean', 
                         'attdruguse_avg':'mean', 
                         'stigma_avg':'mean', 
                         'alienation_avg':'mean', 
                         'deval_avg':'mean', 
                         'pwudserv_avg':'mean', 
                         'pwudrecovery_avg':'mean'})[col_name], 
            text=trend.groupby('year').agg({'num_of_participants':'count', 
                         'issuehelp_ttl':'mean', 
                         'cdrugprob_ttl':'mean', 
                         'trust_avg':'mean', 
                         'attdruguse_avg':'mean', 
                         'stigma_avg':'mean', 
                         'alienation_avg':'mean', 
                         'deval_avg':'mean', 
                         'pwudserv_avg':'mean', 
                         'pwudrecovery_avg':'mean'})[col_name].values.tolist(), 
            name=col_name,
            visible = False
            )
        )
  
for i, column in enumerate(trend.set_index('year').columns): # set index so that they don't display
  args = [False] * len(trend.set_index('year').columns)
  args[i] = True

  button4 = dict(label = column,
                  method = "update",
                  args=[{"visible": args},
                        {"annotations": list(d4.values())[i]}])
  buttons4.append(button4)


fig4.update_layout(
    updatemenus=[dict(active=0,
                      type="dropdown",
                    buttons=buttons4,
                    x = 1,
                    y = 1.1,
                    xanchor = 'right',
                    yanchor = 'bottom'
                )], 
    margin=dict(l=20, r=20, t=60, b=20),#setup the margin
    paper_bgcolor="aliceblue", #setup the background color
    title_text="Yearly Trend (does not change with the year filter)",
    title_x=0.5,
    showlegend=False,
    title_font_size=22)

fig4.show()

### Reliability measure

This table returns reliability of the questionaire for each year.

In [ ]:
# select columns
col_name = ['cdrugprob','ctool','trust', 'worry', 'althiv','cedr','com.*up','self','cabsat','cabactivity', '\Ace_','Stigma','Alienation','deval','pwudserv','pwudrecovery','cad_']

In [ ]:
# define function to output CA table
def reliability_tbl(df):
  table = pd.DataFrame(columns=['Questions', 'Cronbach\'s Alpha','Number of items','95% CI-low','95% CI-high']) # you can add more columns needed
  list1 = []
  list2 = []
  list3 = []
  list4 = []
  list5 = []
  
  # iterate over column to obtain wanted data
  for i in col_name:
    list1.append(num.loc[:, df.columns.str.contains(i)].columns.tolist()) # this return column name
    list2.append(pg.cronbach_alpha(data=df.loc[:, num.columns.str.contains(i)])[0]) # this returns CA for each set of questions
    list3.append(len(num.loc[:, df.columns.str.contains(i)].columns)) # this return the total number of questions contained in one set
    list4.append(pg.cronbach_alpha(data=df.loc[:, num.columns.str.contains(i)])[1].tolist()[0]) # this returns the low value for 95% CI
    list5.append(pg.cronbach_alpha(data=df.loc[:, num.columns.str.contains(i)])[1].tolist()[1]) # this returns the high value for 95% CI
  
  # fill the columns
  table['Questions'] = list1
  table['Cronbach\'s Alpha'] = list2
  table['Number of items'] = list3
  table['95% CI-low'] = list4
  table['95% CI-high'] = list5

  #convert list to string to be better displayed in dash
  table['Questions'] = [','.join(map(str, l)) for l in table['Questions']] 

  return table

# Dashboard building

### import packages

In [ ]:
!pip install dash_bootstrap_components
!pip install jupyter-dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc
import dash_table as dt

## build dash

### Dash layout, for website

documentation here: https://dash.plotly.com/layout

In [ ]:
app = JupyterDash()
app.layout = html.Div([
            ### Header   
            html.H1(
                    children="Community Advisory Board (CAB) Analytics", style={'textAlign': 'center'}, className="header-title" 
                ), # Header title

            html.H2(
                    children="Analyze the CAB response by year between 2020 and 2022", className="header-description", style={'textAlign': 'center'},
                ), # Sub-header

            

            ### Dropdown Menu
            html.Div(className="row", children=[
            # year drop-down
            html.Div(className='two columns', children=[
                html.P('Select Year:', className = 'fix_label', style = {'color': 'black'}),
                dcc.Dropdown(
                    options=[{'label':year, 'value':year} for year in result.year.dropna().unique()] + [{'label': 'Select all', 'value': 'all_year'}], 
                    value=['all_year'],  
                    multi=True, 
                    id = 'year-filter',
                    clearable = True,
                    disabled = False,
                    searchable = True,
                    style = {'display': True},
                    placeholder = 'Select Year',
                    className = 'dropdown',
                )], style=dict(width='50%')), 
            
            # state dropdown
            html.Div(className='two columns', children=[
                html.P('Select State:', className = 'fix_label', style = {'color': 'black'}),
                dcc.Dropdown(id = 'state-filter',
                    options=[{'label':state, 'value':state} for state in result.state.dropna().unique()] + [{'label': 'Select all', 'value': 'all_state'}], 
                    value=['all_state'],  
                    multi=True,
                    clearable = True,
                    disabled = False,
                    searchable = True,
                    style = {'display': True},
                    placeholder = 'Select State',
                    className = 'dropdown',
                )], style=dict(width='50%'))
            ],
            style=dict(display='flex')),
            
            
            ### Graphs
            html.H2(children="Graphs",
                    className="header-description", style={'textAlign': 'center'}), # Note
            html.H3(
                    children="* If the graph is not displaying, that means the question was not asked that year *",
                    className="header-description", style={'textAlign': 'center'},
                ), # Note
            html.H3(
                    children="* If after the reselection of the values, the graph is not changing, that means the value does not exist for the year(s) you select *",
                    className="header-description", style={'textAlign': 'center'},
                ), # Note
            
            # first two graphs
            html.Div(children=[
            dcc.Graph(id='fig1', figure = fig1, style={'width': '50%','display': 'inline-block'}),
            dcc.Graph(id='fig2', figure = fig2, style={'width': '50%','display': 'inline-block'})
            ]),

            # second two graphs
            html.Div(children=[
            dcc.Graph(id='fig3', figure = fig3, style={'width': '50%','display': 'inline-block'}),
            dcc.Graph(id='fig4', figure = fig4, style={'width': '50%','display': 'inline-block'})
            ]),
          
            ### Table
            html.Center(children=[
            html.H2(children="Table",
                    className="header-description", style={'textAlign': 'center'}), # Note
            html.H3(children="* If column is blank, that means this set of questions was not asked that year. The calculation should be based on one year. *",
                    className="header-description", style={'textAlign': 'center'}), # Note

            dt.DataTable(style_data={
                'whiteSpace': 'normal',
                'height': 'auto'},
                id='tbl', 
                data=reliability_tbl(num).to_dict('records'),
                columns=[{"name": i, "id": i} for i in reliability_tbl(num).columns],
                style_table={'overflowX': 'auto'},
                style_cell={
                    'height': 'auto',
                    # all three widths are needed
                    'minWidth': '20px', 'width': '20px', 'maxWidth': '380px',
                    'whiteSpace': 'normal'})
                    ])
            ])

### Callback, to make the dropdown selection function.

You can see the documentation here: https://dash.plotly.com/basic-callbacks You need to click the link the section produces to see the results

In [ ]:
###fig 1
@app.callback(
    Output("fig1", "figure"), #the output is the figure
    Input("year-filter", "value"),#the input is the year-filter
    Input("state-filter", "value"))# the input is the state-filter

#Figure Update
def update_figure(year,state):
  # this updates the data
  filtered_data = result

  if year == ['all_year']:
    filtered_data = filtered_data
  else:
    filtered_data = filtered_data[filtered_data["year"].isin(year)]

  if state == ['all_state']:
    filtered_data = filtered_data
  else:
    filtered_data = filtered_data[filtered_data["state"].isin(state)]
  
  # from here, it is copy-pasting the code in figure 1
  survey = filtered_data[['Progress_group','Finished']]
  survey = survey.apply(lambda x: x.sort_values().values)

  fig1 = go.Figure()
  buttons = []

  for col_name in survey.columns:
  ## add traces
    fig1.add_trace(go.Bar(
              x=survey[col_name].unique(),
              y=survey[col_name].value_counts(),
              text=survey[col_name].value_counts(),  
              name=col_name,
              visible = False
              )
          )

  fig1.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=list([
                dict(label="Progress_group",
                      method="update",
                      args=[{"visible": [True, False]},
                            {"annotations": anno1}]),
                dict(label="Finished",
                      method="update",
                      args=[{"visible": [False, True]},
                            {"annotations": anno2}])
            ]),
              type="dropdown",
              x = 1,
              y = 1.1,
              xanchor = 'right',
              yanchor = 'bottom'
        )     
    ],
    margin=dict(l=20, r=20, t=60, b=20),#setup the margin
    paper_bgcolor="aliceblue", #setup the background color
    title_text="Survey Completion Status",
    title_x=0.5,
    showlegend=False,
    title_font_size=22)

  return fig1 #return the fig according to the filter

### fig2
@app.callback(
    Output("fig2", "figure"), #the output is the chart
    Input("year-filter", "value"),#the input is the year-filter
    Input("state-filter", "value"))

#Figure Update
def update_figure(year,state):
  # this updates the data
  filtered_data = result

  if year == ['all_year']:
    filtered_data = filtered_data
  else:
    filtered_data = filtered_data[filtered_data["year"].isin(year)]

  if state == ['all_state']:
    filtered_data = filtered_data
  else:
    filtered_data = filtered_data[filtered_data["state"].isin(state)]

  # same as fig2
  demo = filtered_data[['join_yr', 'inst', 'membdrug', 'othboar', 'leadoth', 'edu','hisp','race','sex',
               'age_group', 'hoursvol_group', 'netorgnum_group','netoutorg_group','internetfreq_group',
               'cabsat_1','cabsat_2','cabsat_3', 'cabactivity_1','cabactivity_2', 'cabactivity_3']]
  demo = demo.apply(lambda x: x.sort_values().values)

  fig2 = go.Figure()

  buttons2 = []

  for col_name in demo.columns:
    ## add traces
    fig2.add_trace(go.Bar(
              x=demo[col_name].unique(),
              y=demo[col_name].value_counts(), 
              text=demo[col_name].value_counts(),  
              name=col_name,
              visible = False
              )
          )

  for i, column in enumerate(demo.columns):
    args = [False] * len(demo.columns)
    args[i] = True

    button2 = dict(label = column,
                    method = "update",
                    args=[{"visible": args},
                          {"annotations": list(d.values())[i]}])
    buttons2.append(button2)

  fig2.update_layout(
      title_text="Participant Information",
      title_x=0.5,
      showlegend=False,
    title_font_size=22
  ) 

  fig2.update_layout(
      updatemenus=[dict(active=0,
                        type="dropdown",
                      buttons=buttons2,
                      x = 1,
                      y = 1.1,
                      xanchor = 'right',
                      yanchor = 'bottom'
                  )], 
      margin=dict(l=20, r=20, t=60, b=20),#setup the margin
      paper_bgcolor="aliceblue", #setup the background color
  )

  return fig2

### fig 3
@app.callback(
    Output("fig3", "figure"), #the output is the chart
    Input("year-filter", "value"),#the input is the year-filter
    Input("state-filter", "value"))

def update_charts(year,state):
  # update data
  filtered_data1 = result
  filtered_data2 = num

  if year == ['all_year']:
    filtered_data1 = filtered_data1
    filtered_data2 = filtered_data2
  else:
    filtered_data1 = filtered_data1[filtered_data1["year"].isin(year)]
    filtered_data2 = filtered_data2[filtered_data2["year"].isin(year)]
  
  if state == ['all_state']:
    filtered_data1 = filtered_data1
    filtered_data2 = filtered_data2
  else:
    filtered_data1 = filtered_data1[filtered_data1["state"].isin(state)]
    filtered_data2 = filtered_data2[filtered_data2["state"].isin(state)]

  # same as above
  # get text response measurement
  measurement1 = filtered_data1.loc[:, filtered_data1.columns.str.contains("issuehelp_|cdrugprob_|trust_|cad_")].drop(['issuehelp_14_TEXT', 'issuehelp_14_TEXT_TEXT'], axis=1)
  measurement1[['year','state']] = filtered_data1[['year','state']]
  measurement1 = measurement1.reset_index().drop(['index'], axis =1).replace(['Click to write Scale Point 2', 'Click to write Scale Point 3', 'Click to write Scale Point 4'], 'Not Applicable')

  # get numerical response measurement
  measurement2 = filtered_data2.loc[:, filtered_data2.columns.str.contains("_avg|_ttl")]
  measurement2[['year','state']] = filtered_data2[['year','state']]
  measurement2 = measurement2.reset_index().drop(['index'], axis =1)

  # concat two responses for plotting
  measurement = pd.concat([measurement1, measurement2], axis = 1)
  measurement = measurement.loc[:, ~measurement.columns.duplicated()]

  fig3 = go.Figure()
  buttons3 = []
  for col_name in measurement1.set_index(['year','state']).columns.to_list():
    ## add traces
    fig3.add_trace(go.Pie(
              values=measurement1.set_index(['year','state'])[col_name].value_counts(), 
              labels=measurement1.set_index(['year','state'])[col_name].value_counts().index.tolist(),
              textinfo='label+percent',
              visible=False,
              )
    )

  for col_name in measurement2.set_index(['year','state']).columns:
    ## add traces
    fig3.add_trace(go.Box(
              y=measurement2.set_index(['year','state'])[col_name].values.tolist(),
              text=measurement2.set_index(['year','state'])[col_name].values.tolist(),  
              name=col_name,
      boxpoints='all',
      jitter=0.3,
      marker = dict(
          color = 'rgb(214,12,140)',
      ),
              visible=False
              )
          )

  for i, column in enumerate(measurement.set_index(['year','state']).columns):
    args = [False] * len(measurement.set_index(['year','state']).columns)
    args[i] = True

    button3 = dict(label = column,
                    method = "update",
                    args=[{"visible": args},
                          {"annotations": list(d2.values())[i]}])
    buttons3.append(button3)

  fig3.update_layout(
      title_text="Key Measurement",
      title_x=0.5,
      showlegend=False,
      title_font_size=22
  ) 

  fig3.update_layout(
      updatemenus=[dict(active=0,
                        type="dropdown",
                      buttons=buttons3,
                      x = 1,
                      y = 1.1,
                      xanchor = 'right',
                      yanchor = 'bottom'
                  )], 
      margin=dict(l=20, r=20, t=60, b=20),#setup the margin
      paper_bgcolor="aliceblue", #setup the background color
  )

  return fig3

### fig 4
@app.callback(
    Output("fig4", "figure"),
    #[Input("year-filter", "value"), # this figure does not change with the year filter, so no need to put input
    [Input("state-filter", "value")])

def update_charts(state):
  # update data
  filtered_data = num

  if state == ['all_state']:
    filtered_data = num
  else:
    filtered_data = num[num["state"].isin(state)]
  
  # select columns
  trend = filtered_data[['year','Status','issuehelp_ttl', 'cdrugprob_ttl', 'trust_avg', 'attdruguse_avg', 'stigma_avg', 'alienation_avg', 'deval_avg', 'pwudserv_avg', 'pwudrecovery_avg']]
  trend = trend.rename({'Status':'num_of_participants'}, axis= 1)

  # same as above
  fig4 = go.Figure()

  buttons4 = []

  for col_name in trend.set_index('year').columns:
    ## add traces
    fig4.add_trace(go.Scatter(
              x=[str(x) for x in trend.year.unique().tolist()],
              # add different aggregations to the figure
              y=trend.groupby('year').agg({'num_of_participants':'count', 
                          'issuehelp_ttl':'mean', 
                          'cdrugprob_ttl':'mean', 
                          'trust_avg':'mean', 
                          'attdruguse_avg':'mean', 
                          'stigma_avg':'mean', 
                          'alienation_avg':'mean', 
                          'deval_avg':'mean', 
                          'pwudserv_avg':'mean', 
                          'pwudrecovery_avg':'mean'})[col_name], 
              text=trend.groupby('year').agg({'num_of_participants':'count', 
                          'issuehelp_ttl':'mean', 
                          'cdrugprob_ttl':'mean', 
                          'trust_avg':'mean', 
                          'attdruguse_avg':'mean', 
                          'stigma_avg':'mean', 
                          'alienation_avg':'mean', 
                          'deval_avg':'mean', 
                          'pwudserv_avg':'mean', 
                          'pwudrecovery_avg':'mean'})[col_name].values.tolist(), 
              name=col_name,
              visible = False
              )
          )
    
  for i, column in enumerate(trend.set_index('year').columns): # set index so that they don't display
    args = [False] * len(trend.set_index('year').columns)
    args[i] = True

    button4 = dict(label = column,
                    method = "update",
                    args=[{"visible": args},
                          {"annotations": list(d4.values())[i]}])
    buttons4.append(button4)


  fig4.update_layout(
      updatemenus=[dict(active=0,
                        type="dropdown",
                      buttons=buttons4,
                      x = 1,
                      y = 1.1,
                      xanchor = 'right',
                      yanchor = 'bottom'
                  )], 
      margin=dict(l=20, r=20, t=60, b=20),#setup the margin
      paper_bgcolor="aliceblue", #setup the background color
      title_text="Yearly Trend (does not change with the year filter)",
      title_x=0.5,
      showlegend=False,
      title_font_size=22)

  fig4.show()

  return fig4

### table
@app.callback(
    Output("tbl", "data"),
    [Input("year-filter", "value")])#the input is the year-filter
    #Input("state-filter", "value")]) # does not need state-filter

def display_table(year):
  filtered_data = num

  if year == ['all_year']:
    filtered_data = num
  else:
    filtered_data = num[num["year"].isin(year)]

  col_name = ['cdrugprob','ctool','trust', 'worry', 'althiv','cedr','com.*up','self','cabsat','cabactivity', '\Ace_','Stigma','Alienation','deval','pwudserv','pwudrecovery','cad_']
  return reliability_tbl(filtered_data).to_dict("records")

if __name__ =='__main__':
  app.run_server(mode = 'external')

Dash app running on:


<IPython.core.display.Javascript object>